In [122]:
import os
from os import environ
from selenium import webdriver
from selenium.webdriver.support.select import Select
import dateparser
import datetime

driver = webdriver.Chrome()

In [123]:
task_type = 'taskCID' # can be 'taskRID' for realID
name = 'First'
last_name = 'Last'
phone_number = '6280000000'

In [124]:
driver.get("https://www.dmv.ca.gov/wasapp/foa/clear.do?goTo=officeVisit&localeName=en")

In [125]:
task_selector = driver.find_element_by_id('one_task')
task_selector.click()

In [126]:
task_type_selector = driver.find_element_by_id(task_type)
if not task_type_selector.is_selected():
    task_type_selector.click() 

In [127]:
name_field = driver.find_element_by_name('firstName')
name_field.clear()
name_field.send_keys(name)
lastname_field = driver.find_element_by_name('lastName')
lastname_field.clear()
lastname_field.send_keys(last_name)

In [128]:
phone_area_field = driver.find_element_by_name('telArea')
phone_area_field.send_keys(phone_number)

In [ ]:
results = {}

In [137]:
selector_field = driver.find_element_by_name('officeId')
offices_count = len(Select(selector_field).options)
for idx in range(44, offices_count):
    try:
        selector_field = driver.find_element_by_name('officeId')
        Select(selector_field).select_by_index(idx)
        submit = driver.find_element_by_class_name('btn-primary')
        submit.click()

        table_rows = driver.find_elements_by_css_selector('tbody > tr > td > p.no-margin-bottom')
        address = table_rows[0].text
        if ('not available' in address) or ('no appointment is available' in address):
            print("Skipping " + str(idx) + " - the service not available there.")
        else:
            datetime_str = table_rows[2].text
            datetime_val = dateparser.parse(datetime_str)
            results[address] = datetime_val

        driver.back()
    except:
        print("Problem processing element " + str(idx))
        raise

Skipping 45 - the service not available there.
Skipping 47 - the service not available there.
Skipping 57 - the service not available there.
Problem processing element 75


IndexError: list index out of range

In [138]:
sorted([date.isoformat() + ' ' + address for address, date in results.items()])

['2018-03-29T09:40:00 GRANADA HILLS DLPC\n16201 San Fernando Mission Blvd\nGRANADA HILLS DLPC, CA',
 '2018-04-05T09:40:00 LAKE ISABELLA\n5520 Lake Isabella Road G1\nLAKE ISABELLA, CA',
 '2018-04-05T14:20:00 LAKEPORT\n965 Parallel Drive\nLAKEPORT, CA',
 '2018-04-09T11:00:00 MADERA\n1206 W. Maple St.\nMADERA, CA',
 '2018-04-12T09:30:00 INDIO\n44480 Jackson St.\nINDIO, CA',
 '2018-04-12T11:00:00 GARBERVILLE\n1180 Evergreen Road Suite A\nGARBERVILLE, CA',
 '2018-04-18T10:00:00 GOLETA\n7127 Hollister Ave. Ste. 24/26\nGOLETA, CA',
 '2018-04-18T10:00:00 LOS GATOS\n600 North Santa Cruz Ave.\nLOS GATOS, CA',
 '2018-04-18T13:00:00 HANFORD\n701 W. Hanford Armona Road\nHANFORD, CA',
 '2018-04-20T10:50:00 LOS BANOS\n814 West L St.\nLOS BANOS, CA',
 '2018-04-20T12:40:00 LOMPOC\n1601 N H St\nLOMPOC, CA',
 '2018-04-25T09:40:00 HEMET\n1200 S. State Street\nHEMET, CA',
 '2018-04-25T13:50:00 KING CITY\n101 East San Antonio Dr.\nKING CITY, CA',
 '2018-04-26T13:10:00 GRASS VALLEY\n435 Sutton Way\nGRASS VAL

In [ ]:
driver.close()